# Analyzing the best Neighborhoods to open an Indian Restaurant chain.


Download all the dependencies that will be needed.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  23.16 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  37.35 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  52.89 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  30.31 MB/s
vincent-0.4.4- 100% |###################

<a id='item1'></a>

## 1. Download and Explore New York Data Set


Luckily, the dataset which contains all the **New York Borouge and Neighbourhood exists for free on the web. Link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572





Downloading the data from the server

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


#### Load and explore the data

Next, let's load the data.

In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Let's take a quick look at the data.

In [4]:
newyork_data

{'bbox': [-74.2492599487305,
  40.5033187866211,
  -73.7061614990234,
  40.9105606079102],
 'crs': {'properties': {'name': 'urn:ogc:def:crs:EPSG::4326'}, 'type': 'name'},
 'features': [{'geometry': {'coordinates': [-73.84720052054902,
     40.89470517661],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.1',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661],
    'borough': 'Bronx',
    'name': 'Wakefield',
    'stacked': 1},
   'type': 'Feature'},
  {'geometry': {'coordinates': [-73.82993910812398, 40.87429419303012],
    'type': 'Point'},
   'geometry_name': 'geom',
   'id': 'nyu_2451_34572.2',
   'properties': {'annoangle': 0.0,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.874294193

We notice all the relevant data is in the *features* key, which is basically a list of the neighborhoods. So, I defined a new variable that includes this data.

In [5]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.

In [6]:
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

#### Tranform the data into a *pandas* dataframe

As the data is in Json format transfroming the data to *Pandas* DataFrame

In [7]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods # Displaying the empty dataframe

,Borough,Neighborhood,Latitude,Longitude


Filling the DF by looping through data

In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe we can see the Unique Borough

In [9]:
neighborhoods.Borough.unique()

array(['Bronx', 'Manhattan', 'Brooklyn', 'Queens', 'Staten Island'], dtype=object)

Checking the shape of the DF.

In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### Now that we have the Neighborhood name we can get the Latitude and longitude data with the help of geopy library.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [11]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7308619, -73.9871558.


#### Create a map of New York with neighborhoods superimposed on top.

In [12]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

By Doing a quick google search I found out that Manhattan is the very famous Borough.

Slice the original dataframe and create a new dataframe of the Manhattan data by simplyfying the above map and segment and cluster only the neighborhoods in Manhattan.

In [13]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
display(manhattan_data.head()) #
display(manhattan_data.shape) #Gives us a count of Neighbourhoods.

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


(40, 4)

#### Using the Geopy library getting the geographical coordinates of Manhattan.

In [14]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


#### Displaying Manhattan neighborhoods on the map using the Folium library.

In [15]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

Now as I have the Data cleaned and I am ready to explore using Foursquare data.

#### Defining Foursquare Credentials and Version

In [16]:
CLIENT_ID = 'VFOAQLPDH5LWOJF5Q4TFDFPCI2U32ZWZWKCKKYI4DX5MRUTG' # your Foursquare ID
CLIENT_SECRET = 'TCX45H5U0YPLJW3SZTOOJEFXQRFT4CYVVRUP45WWCAFCHF2Y' # your Foursquare Secret
VERSION = '20190303' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: VFOAQLPDH5LWOJF5Q4TFDFPCI2U32ZWZWKCKKYI4DX5MRUTG
CLIENT_SECRET:TCX45H5U0YPLJW3SZTOOJEFXQRFT4CYVVRUP45WWCAFCHF2Y


#### Exploring just one Neighbourhood to figure out how we can proceed with the data which we get by calling the API.

Getting the first neighborhood's name.

In [17]:
manhattan_data.loc[0, 'Neighborhood']

'Marble Hill'

Getting the neighborhood's latitude and longitude values.

In [18]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


#### Now, using the Foursquare API to get the top 100 venues that are in Marble Hill within a radius of 500 meters.

First I created the URL to send for the GET result

In [19]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL




'https://api.foursquare.com/v2/venues/explore?&client_id=VFOAQLPDH5LWOJF5Q4TFDFPCI2U32ZWZWKCKKYI4DX5MRUTG&client_secret=TCX45H5U0YPLJW3SZTOOJEFXQRFT4CYVVRUP45WWCAFCHF2Y&v=20190303&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

Sending the GET request and examine the resutls

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c8aa99bdd57977684a56bba'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b4429abf964a52037f225e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1ca941735',
         'name': 'Pizza Place',
         'pluralName': 'Pizza Places',
         'primary': True,
         'shortName': 'Pizza'}],
       'delivery': {'id': '72548',
        'provider': {'icon': {'name': '/delivery_provider_seamless_20180129.png',
          'prefix': 'https://fastly.4sqi.net/img/general/cap/',
          'sizes': [40, 50]},
         'name': 'seamless'},
        'url': 'https://www.seamless.com/menu/arturos-pizza-5189-broadway-ave-new-york/72548?affiliate=1131&utm_source=foursquare-affiliat

As the information is in the *items* key. Before i proceed, I borrowed the **get_category_type** function from the Foursquare lab. Which basically gives the catogory of Venues returned.

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now Cleaning the Json file whihc was returned and saving it into a DF.

In [22]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Starbucks,Coffee Shop,40.877531,-73.905582
4,Dunkin' Donuts,Donut Shop,40.876993,-73.906507


In [23]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

24 venues were returned by Foursquare.


Eventhough we called limited the venues to 100 we just got 24 Venues from that Neighborhood.

<a id='item2'></a>

## Exploring all Neighborhoods in Manhattan now.

#### Creating a function to repeat the same process to all the neighborhoods in Manhattan

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Writing the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [25]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )



Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


investigating the data

In [26]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3301, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin' Donuts,40.876993,-73.906507,Donut Shop


By checking how many venues were returned for each neighborhood

In [27]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,99,99,99,99,99,99
Carnegie Hill,100,100,100,100,100,100
Central Harlem,44,44,44,44,44,44
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,45,45,45,45,45,45
East Village,100,100,100,100,100,100


#### finding out how many unique categories can be curated from all the returned venues

In [28]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 329 uniques categories.


As there are 330 uniques categories now I am trying to figure out whats the most common Vanues and the number of Venues for those unique vaules and We find out there are 30 Indian Restaurants in Mahattan Neighborhoods

In [29]:
manhattan_venues['Venue Category'].value_counts()

Coffee Shop                                 130
Italian Restaurant                          129
Bakery                                       78
Café                                         76
American Restaurant                          75
Pizza Place                                  68
Park                                         64
Hotel                                        63
Gym                                          60
Mexican Restaurant                           59
Bar                                          58
French Restaurant                            57
Gym / Fitness Center                         57
Cocktail Bar                                 52
Sushi Restaurant                             47
Spa                                          47
Chinese Restaurant                           47
Sandwich Place                               46
Wine Shop                                    44
Japanese Restaurant                          43
Clothing Store                          

## Analyzing Each Neighborhood to figure out the top 10 most common venues.

In [30]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Social Cl

By Examining the data shape

In [31]:
manhattan_onehot.shape

(3301, 330)

#### By grouping the rows of neighborhood and by taking the mean of the frequency of occurrence of each category

In [32]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soba Restaurant,Social Cl

Now the new shape give us the Neighbourhoods with mean of each venues

In [33]:
manhattan_grouped.shape

(40, 330)

#### By printing each neighborhood along with the top 5 most frequent venues we can visualize the top 5

In [34]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                venue  freq
0                Park  0.08
1         Coffee Shop  0.08
2               Hotel  0.05
3                 Gym  0.03
4  Italian Restaurant  0.03


----Carnegie Hill----
               venue  freq
0        Pizza Place  0.06
1        Coffee Shop  0.06
2               Café  0.04
3                Bar  0.04
4  French Restaurant  0.03


----Central Harlem----
                  venue  freq
0    African Restaurant  0.07
1     French Restaurant  0.05
2    Chinese Restaurant  0.05
3   American Restaurant  0.05
4  Gym / Fitness Center  0.05


----Chelsea----
                 venue  freq
0          Coffee Shop  0.07
1   Italian Restaurant  0.06
2       Ice Cream Shop  0.05
3               Bakery  0.04
4  American Restaurant  0.04


----Chinatown----
                   venue  freq
0     Chinese Restaurant  0.09
1     Dim Sum Restaurant  0.04
2    American Restaurant  0.04
3  Vietnamese Restaurant  0.04
4           Cocktail Bar  0.04


----Civic Cente

Sorting the venues in descending order by frequency

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Creating the new dataframe and displaying the top 100 venues for each neighborhood.

In [38]:
num_top_venues = 100

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue,51th Most Common Venue,52th Most Common Venue,53th Most Common Venue,54th Most Common Venue,55th Most Common Venue,56th Most Common Venue,57th Most Common Venue,58th Most Common Venue,59th Most Common Venue,60th Most Common Venue,61th Most Common Venue,62th Most Common Venue,63th Most Common Venue,64th Most Common Venue,65th Most Common Venue,66th Most Common Venue,67th Most Common Venue,68th Most Common Venue,69th Most Common Venue,70th Most Common Venue,71th Most Common Venue,72th Most Common Venue,73th Most Common Venue,74th Most Common Venue,75th Most Common Venue,76th Most Common Venue,77th Most Common Venue,78th Most Common Venue,79th Most Common Venue,80th Most Common Venue,81th Most Common Venue,82th Most Common Venue,83th Most Common Venue,84th Most Common Venue,85th Most Common Venue,86th Most Common Venue,87th Most Common Venue,88th Most Common Venue,89th Most Common Venue,90th Most Common Venue,91th Most Common Venue,92th Most Common Venue,93th Most Common Venue,94th Most Common Venue,95th Most Common Venue,96th Most Common Venue,97th Most Common Venue,98th Most Common Venue,99th Most Common Venue,100th Most Common Venue
0,Battery Park City,Coffee Shop,Park,Hotel,Gym,Wine Shop,Italian Restaurant,Shopping Mall,Fountain,BBQ Joint,Plaza,Food Truck,Food Court,Burger Joint,Cupcake Shop,Pizza Place,Memorial Site,Ice Cream Shop,Sandwich Place,Clothing Store,Department Store,Market,Steakhouse,Garden,Lingerie Store,Burrito Place,Mediterranean Restaurant,Building,Historic Site,Men's Store,Mexican Restaurant,Smoke Shop,Boat or Ferry,Candy Store,Seafood Restaurant,Sculpture Garden,Scenic Lookout,Chinese Restaurant,Movie Theater,Salad Place,Electronics Store,Bistro,Sushi Restaurant,Juice Bar,Performing Arts Venue,American Restaurant,Pet Store,Athletics & Sports,Auditorium,Tree,Women's Store,Grocery Store,Public Art,Gourmet Shop,Bakery,Monument / Landmark,Beer Garden,Gastropub,Pub,Cosmetics Shop,Playground,Eastern European Restaurant,Diner,Discount Store,Dive Bar,Fast Food Restaurant,Dog Run,Falafel Restaurant,Donut Shop,English Restaurant,Ethiopian Restaurant,Drugstore,Event Space,Exhibit,Duty-free Shop,Dumpling Restaurant,Farmers Market,French Restaurant,Filipino Restaurant,Fish Market,High School,Herbs & Spices Store,Heliport,Health Food Store,Health & Beauty Service,Hawaiian Restaurant,Hardware Store,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center,Greek Restaurant,Golf Course,Gluten-free Restaurant,Gift Shop,German Restaurant,General Entertainment,General College & University,Gay Bar,Gas Station,Garden Center
1,Carnegie Hill,Coffee Shop,Pizza Place,Café,Bar,Japanese Restaurant,French Restaurant,Gym,Yoga Studio,Bookstore,Cosmetics Shop,Wine Shop,Spa,Pub,Vietnamese Restaurant,Grocery Store,Bakery,Supermarket,Gym / Fitness Center,Italian Restaurant,Sports Bar,Community Center,Breakf

<a id='item4'></a>

Let's check if the top 100 Venues has Indian Restaurant or not and drop the once which already has Indian.

In [56]:
#From 40 Neighbourhoods to just 20 Neighbourhoods which doesnt even have Indian Restaurants.
neighborhoods_venues_woindian_restaurant = neighborhoods_venues_sorted[neighborhoods_venues_sorted.iloc[:, 0:] != "Indian Restaurant"]
neighborhoods_venues_woindian_restaurant.dropna(axis=0, inplace=True)
display(neighborhoods_venues_woindian_restaurant)
display(neighborhoods_venues_woindian_restaurant.shape)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue,41th Most Common Venue,42th Most Common Venue,43th Most Common Venue,44th Most Common Venue,45th Most Common Venue,46th Most Common Venue,47th Most Common Venue,48th Most Common Venue,49th Most Common Venue,50th Most Common Venue,51th Most Common Venue,52th Most Common Venue,53th Most Common Venue,54th Most Common Venue,55th Most Common Venue,56th Most Common Venue,57th Most Common Venue,58th Most Common Venue,59th Most Common Venue,60th Most Common Venue,61th Most Common Venue,62th Most Common Venue,63th Most Common Venue,64th Most Common Venue,65th Most Common Venue,66th Most Common Venue,67th Most Common Venue,68th Most Common Venue,69th Most Common Venue,70th Most Common Venue,71th Most Common Venue,72th Most Common Venue,73th Most Common Venue,74th Most Common Venue,75th Most Common Venue,76th Most Common Venue,77th Most Common Venue,78th Most Common Venue,79th Most Common Venue,80th Most Common Venue,81th Most Common Venue,82th Most Common Venue,83th Most Common Venue,84th Most Common Venue,85th Most Common Venue,86th Most Common Venue,87th Most Common Venue,88th Most Common Venue,89th Most Common Venue,90th Most Common Venue,91th Most Common Venue,92th Most Common Venue,93th Most Common Venue,94th Most Common Venue,95th Most Common Venue,96th Most Common Venue,97th Most Common Venue,98th Most Common Venue,99th Most Common Venue,100th Most Common Venue
0,Battery Park City,Coffee Shop,Park,Hotel,Gym,Wine Shop,Italian Restaurant,Shopping Mall,Fountain,BBQ Joint,Plaza,Food Truck,Food Court,Burger Joint,Cupcake Shop,Pizza Place,Memorial Site,Ice Cream Shop,Sandwich Place,Clothing Store,Department Store,Market,Steakhouse,Garden,Lingerie Store,Burrito Place,Mediterranean Restaurant,Building,Historic Site,Men's Store,Mexican Restaurant,Smoke Shop,Boat or Ferry,Candy Store,Seafood Restaurant,Sculpture Garden,Scenic Lookout,Chinese Restaurant,Movie Theater,Salad Place,Electronics Store,Bistro,Sushi Restaurant,Juice Bar,Performing Arts Venue,American Restaurant,Pet Store,Athletics & Sports,Auditorium,Tree,Women's Store,Grocery Store,Public Art,Gourmet Shop,Bakery,Monument / Landmark,Beer Garden,Gastropub,Pub,Cosmetics Shop,Playground,Eastern European Restaurant,Diner,Discount Store,Dive Bar,Fast Food Restaurant,Dog Run,Falafel Restaurant,Donut Shop,English Restaurant,Ethiopian Restaurant,Drugstore,Event Space,Exhibit,Duty-free Shop,Dumpling Restaurant,Farmers Market,French Restaurant,Filipino Restaurant,Fish Market,High School,Herbs & Spices Store,Heliport,Health Food Store,Health & Beauty Service,Hawaiian Restaurant,Hardware Store,Harbor / Marina,Gymnastics Gym,Gym Pool,Gym / Fitness Center,Greek Restaurant,Golf Course,Gluten-free Restaurant,Gift Shop,German Restaurant,General Entertainment,General College & University,Gay Bar,Gas Station,Garden Center
2,Central Harlem,African Restaurant,American Restaurant,French Restaurant,Gym / Fitness Center,Chinese Restaurant,Seafood Restaurant,Public Art,Pizza Place,Bookstore,Library,Music Venue,Ethiopian Restaurant,Market,Gym,Event Space,Metro Station,Boutique,Park,Caribbean Restaur

(20, 101)

#### Now that i have 20 Neighborhood which doesnt have Indian Restaurant i can just use the top 10 Venues for further analysis(which is to find out which Neighborhood is more centered to Restaurants in the top 10 venues)

In [66]:
Neighborhood_top10_venues = neighborhoods_venues_woindian_restaurant.iloc[:,:11]
Neighborhood_top10_venues.index = range(20)
Neighborhood_top10_venues

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Park,Hotel,Gym,Wine Shop,Italian Restaurant,Shopping Mall,Fountain,BBQ Joint,Plaza
1,Central Harlem,African Restaurant,American Restaurant,French Restaurant,Gym / Fitness Center,Chinese Restaurant,Seafood Restaurant,Public Art,Pizza Place,Bookstore,Library
2,Chinatown,Chinese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,American Restaurant,Cocktail Bar,Ice Cream Shop,Hotpot Restaurant,Salon / Barbershop,Noodle House,Bakery
3,Clinton,Theater,Gym / Fitness Center,Hotel,American Restaurant,Indie Theater,Gym,Italian Restaurant,Coffee Shop,Spa,Wine Shop
4,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Latin American Restaurant,Thai Restaurant,Pharmacy,Street Art,Cocktail Bar,Clothing Store,Pet Store
5,East Village,Bar,Ice Cream Shop,Wine Bar,Mexican Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Coffee Shop,Cocktail Bar,Ramen Restaurant,Pizza Place
6,Gramercy,Italian Restaurant,Bagel Shop,Thrift / Vintage Store,Pizza Place,Coffee Shop,Cocktail Bar,Restaurant,Mexican Restaurant,Bar,Grocery Store
7,Hudson Yards,Italian Restaurant,Coffee Shop,Restaurant,Gym / Fitness Center,American Restaurant,Café,Hotel,Theater,Dog Run,Art Gallery
8,Inwood,Mexican Restaurant,Café,Pizza Place,Lounge,Restaurant,Frozen Yogurt Shop,Chinese Restaurant,Coffee Shop,Bakery,Deli / Bodega
9,Lincoln Square,Theater,Gym / Fitness Center,Café,Italian Restaurant,Plaza,Concert Hall,Opera House,French Restaurant,Performing Arts Venue,Park


In [97]:
Neighborhood_top10_venues_copy = Neighborhood_top10_venues
Neighborhood_top10_venues["Total_Restaurants"] = 0

for c in range(20):
    a = Neighborhood_top10_venues.loc[c,:]
    b = a.tolist()
    for y in b:
        if  "Restaurant" in str(y):
            Neighborhood_top10_venues.loc[c, "Total_Restaurants"] = Neighborhood_top10_venues.loc[c, "Total_Restaurants"] + 1
Neighborhood_top10_venues

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Total_Restaurants
0,Battery Park City,Coffee Shop,Park,Hotel,Gym,Wine Shop,Italian Restaurant,Shopping Mall,Fountain,BBQ Joint,Plaza,1
1,Central Harlem,African Restaurant,American Restaurant,French Restaurant,Gym / Fitness Center,Chinese Restaurant,Seafood Restaurant,Public Art,Pizza Place,Bookstore,Library,5
2,Chinatown,Chinese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,American Restaurant,Cocktail Bar,Ice Cream Shop,Hotpot Restaurant,Salon / Barbershop,Noodle House,Bakery,5
3,Clinton,Theater,Gym / Fitness Center,Hotel,American Restaurant,Indie Theater,Gym,Italian Restaurant,Coffee Shop,Spa,Wine Shop,2
4,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Latin American Restaurant,Thai Restaurant,Pharmacy,Street Art,Cocktail Bar,Clothing Store,Pet Store,3
5,East Village,Bar,Ice Cream Shop,Wine Bar,Mexican Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Coffee Shop,Cocktail Bar,Ramen Restaurant,Pizza Place,4
6,Gramercy,Italian Restaurant,Bagel Shop,Thrift / Vintage Store,Pizza Place,Coffee Shop,Cocktail Bar,Restaurant,Mexican Restaurant,Bar,Grocery Store,3
7,Hudson Yards,Italian Restaurant,Coffee Shop,Restaurant,Gym / Fitness Center,American Restaurant,Café,Hotel,Theater,Dog Run,Art Gallery,3
8,Inwood,Mexican Restaurant,Café,Pizza Place,Lounge,Restaurant,Frozen Yogurt Shop,Chinese Restaurant,Coffee Shop,Bakery,Deli / Bodega,3
9,Lincoln Square,Theater,Gym / Fitness Center,Café,Italian Restaurant,Plaza,Concert Hall,Opera House,French Restaurant,Performing Arts Venue,Park,2


In [106]:
#Filtering Neighborhoods with top 10 venues containing more than 4 Restaurants as the 75th percentile is 4 Restaurant I am aiming more than 4.
display(Neighborhood_top10_venues["Total_Restaurants"].describe())
f = Neighborhood_top10_venues[Neighborhood_top10_venues["Total_Restaurants"] > 4]
display(f)

count    20.000000
mean      2.900000
std       1.209611
min       1.000000
25%       2.000000
50%       3.000000
75%       4.000000
max       5.000000
Name: Total_Restaurants, dtype: float64

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Total_Restaurants
1,Central Harlem,African Restaurant,American Restaurant,French Restaurant,Gym / Fitness Center,Chinese Restaurant,Seafood Restaurant,Public Art,Pizza Place,Bookstore,Library,5
2,Chinatown,Chinese Restaurant,Vietnamese Restaurant,Dim Sum Restaurant,American Restaurant,Cocktail Bar,Ice Cream Shop,Hotpot Restaurant,Salon / Barbershop,Noodle House,Bakery,5


## Displaying the neigborhoods on Map to.

In [120]:
#Getting the lattitude and longitude for both Neighbourhoods
manhattan_data.head()
gh =manhattan_data[manhattan_data["Neighborhood"].str.contains("Chinatown") | manhattan_data["Neighborhood"].str.contains("Central Harlem")]
gh

,Borough,Neighborhood,Latitude,Longitude
1,Manhattan,Chinatown,40.715618,-73.994279
6,Manhattan,Central Harlem,40.815976,-73.943211


In [123]:
#Displaying the Neighborhoods on Manhattan MAP
address = 'Manhattan, NY'

# create map of Manhattan using latitude and longitude values
final_map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(gh['Latitude'], gh['Longitude'], gh['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(final_map_manhattan)  
    
final_map_manhattan

# Concluding that the Restaurant owner can open his two new Indian Restaurants in Chainatown Manhattan and Central Harlem Manhattan